In [1]:
## RAG implemetation

In [ ]:
# !pip install -q pypdf
# !pip install -q faiss-gpu
# !pip install faiss-cpu
# %pip install langchain_experimental langchain_core

In [3]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name="gemini-pro", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


def generate_pro(input_prompt):
    model = GenerativeModel("gemini-pro")
    full_prompt = '''summarize the prompt below and do note prompt below will be send to imagen mode so please clean up any sensitve words and replace them into unblocked words like replace girl or lady can be replaced by female human and so on : ''' + input_prompt
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 8190,
        "temperature": 0.2,
        "top_p": 1
    },stream=False,)
    
    # print (responses.text)
    
    return(responses.text)



In [5]:
from crewai import Agent, Task, Crew, Process
# from langchain_openai import ChatOpenAI
from langchain_core.retrievers import BaseRetriever
# from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import VertexAIEmbeddings


from langchain.tools import tool
from langchain_community.document_loaders import WebBaseLoader
import requests, os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.tools import DuckDuckGoSearchRun

embeddings = embedding_function = VertexAIEmbeddings("textembedding-gecko@003")

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import VertexAIEmbeddings



/opt/conda/envs/r/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.vertexai.VertexAIEmbeddings` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAIEmbeddings`.
  warn_deprecated(


In [6]:
# pages[0].page_content

In [7]:
loader = PyPDFLoader("data/dc-duttas-textbook-of-obstetrics-including-perinatology-and-contraception.pdf") 
#GCP-Machine-learning-Certification.pdf")

pages = loader.load_and_split()
state_of_the_union = pages[0].page_content

from langchain_experimental.text_splitter import SemanticChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter


### experimental Sematic splitter 

In [8]:

text_splitter = SemanticChunker(VertexAIEmbeddings())

# Split Text
docs = text_splitter.create_documents([state_of_the_union])
# print(docs[0].page_content)

faiss_index = FAISS.from_documents(docs, VertexAIEmbeddings())

# query = "what  is Data Leakage"
# docs = faiss_index.similarity_search(query, k=2)
# retriever = ""
# for doc in docs:
#     print( doc.page_content[:300])
#     retriever = doc.page_content

2024-03-30 15:12:30,097 - 140545344485184 - vertexai.py-vertexai:41 - WARNING: Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001
2024-03-30 15:12:30,313 - 140545344485184 - vertexai.py-vertexai:41 - WARNING: Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


In [9]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)

# Index
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

from typing import Dict, TypedDict

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, 
        that contains retrieved documents
    """
    print("---RETRIEVE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    local = state_dict["local"]
    documents = retriever.get_relevant_documents(question)
    return {"keys": {"documents": documents, "local": local, 
            "question": question}}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, 
        that contains LLM generation
    """
    print("---GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # Post-processing
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    generation = rag_chain.invoke({"context": documents, 
                                  "question": question})
    return {
        "keys": {"documents": documents, "question": question, 
                               "generation": generation}
    }


In [10]:
pages[0]

Document(page_content=';\n&\ngynecology\nbooks\n,\nvisit\nour\nwebsite\nwww\n.\njaypeebrothers\n.\ncom\n,\nfor\ndetailed\ninformation\non\nobstetrics\n&\ngynecology\nbooks\nGet\nSupplementary\nVideo\nAccess\nat\nHelp\nus\nin\nfighting\nthe\nPiracy\n.\nLook\nout\nfor\nHologram\nand\nonline\nscratch\ncode\nbefore\nbuying\nthe\nbook\n.\n*\nOBSTETRIC\nOPERATION\n:\nLower\nSegment\nCesarean\nSection\nDC\nDutta\n’\ns\nTextbook\nof\nOBSTETRICS\nHiralal\nKonar\nth\nEdition', metadata={'source': 'data/dc-duttas-textbook-of-obstetrics-including-perinatology-and-contraception.pdf', 'page': 0})

In [13]:


# Tool 1 : Save the news articles in a database
class SearchNewsDB:
    @tool("News DB Tool")
    def news(query: str):
        """Fetch news articles and process their contents."""

        docs = faiss_index.similarity_search(query, k=2)
        retriever = ""
        for doc in docs:
            print(doc.page_content[:300])
            retriever = doc.page_content
            
        # Index the accumulated content splits if there are any
        if retriever:
            # vectorstore = Chroma.from_documents(all_splits, embedding=embedding_function, persist_directory="./chroma_db")
            # retriever = vectorstore.similarity_search(query)
            return retriever
        else:
            return "No content available for processing."

# Tool 2 : Get the news articles from the database
class GetNews:
    @tool("Get News Tool")
    def news(query: str) -> str:
        """Search Chroma DB for relevant information based on a query."""
        vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
        retriever = vectorstore.similarity_search(query)
        return retriever

# Tool 3 : Search for news articles on the web
# search_tool = DuckDuckGoSearchRun()

# 2. Creating Agents
news_search_agent = Agent(
    role='News Seacher',
    goal='Generate key points for each article from the latest docs',
    backstory='Expert in analysing and generating key points from search content for quick updates.',
    tools=[SearchNewsDB().news],
    allow_delegation=True,
    verbose=True,
    llm=llm
)

writer_agent = Agent(
    role='Writer',
    goal='Identify all the topics received. Use the Get Search Tool to verify the each topic to search. Use the Search tool for detailed exploration of each topic. Summarise the retrieved information in depth for every topic.',
    backstory='Expert in crafting engaging narratives from complex information.',
    tools=[GetNews().news],
    allow_delegation=True,
    verbose=True,
    llm=llm
)

# 3. Creating Tasks
news_search_task = Task(
    # description='Search for AI 2024 and create key points for each news.',
    description='what  is hysterectomy.',    
    agent=news_search_agent,
    tools=[SearchNewsDB().news],
    expected_output='A structured email accourding to the instructions'
)

writer_task = Task(
    description="""
    Go step by step.
    Step 1: Identify all the topics received.
    Step 2: Use the Get Search Tool to verify the each topic by going through one by one.
    Step 3: Use the Search tool to search for information on each topic one by one. 
    Step 4: Go through every topic and write an in-depth summary of the information retrieved.
    Don't skip any topic.
    """,
    agent=writer_agent,
    context=[news_search_task],
    tools=[GetNews().news ],
    expected_output='A structured email accourding to the instructions'
)

# 4. Creating Crew
news_crew = Crew(
    agents=[news_search_agent, writer_agent],
    tasks=[news_search_task, writer_task],
    process=Process.sequential, 
    manager_llm=llm,
    expected_output='A structured email accourding to the instructions'
)

# Execute the crew to see RAG in action
result = news_crew.kickoff()
print(result)

2024-03-30 15:16:20,191 - 140545344485184 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed




> Entering new CrewAgentExecutor chain...
Action: News DB Tool
Action Input: {"query": "briefly define hysterectomy"}jaypeebrothers
. com
,
for
detailed
information
on
obstetrics
&
gynecology
books
Get
Supplementary
Video
Access
at
Help
us
in
fighting
the
Piracy
. Look
out
for
Hologram
and
online
scratch
code
before
buying
the
book
. *
OBSTETRIC
OPERATION
:
Lower
Segment
Cesarean
Section
DC
Dutta
’
s
Textbook
of
O
;
&
gynecology
books
,
visit
our
website
www
.
 

;
&
gynecology
books
,
visit
our
website
www
.

Thought: seems like I need to ask a colleague
Action: Delegate work to co-worker
Action Input: {
    'coworker': 'Writer',
    'task': 'Write an email according to the given criteria',
    'context': 'This is a structured email according to the instructions. Please return the actual complete content as the final answer, not a summary. Begin! This is VERY important to you, use the tools available and give your best Final Answer, your job depends on it!'
}

> Entering new CrewAge

ValueError: Content roles do not match: model != 